<a href="https://colab.research.google.com/github/lahmuddin/Machine-Learning-after-Middle-Test/blob/main/Final%20Exam/UAS_ML_Code2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Lahmuddin_1103184028_TK-42-G6**

reproduce from https://github.com/Vedurumudi-Priyanka/Twitter-Sentiment-Analysis

## **Installation**

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [13]:
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

In [14]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import re
import sys
from nltk.stem.porter import PorterStemmer

import random
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import lil_matrix
from sklearn.feature_extraction.text import TfidfTransformer

import nltk
import pickle

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier

from sklearn import svm

from keras.models import Sequential, load_model
from keras.layers import Dense

from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import LSTM
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Embedding, Flatten
from keras.layers import Conv1D, GlobalMaxPooling1D

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import loadtxt
import pickle

import glob

import csv

In [2]:
import sys
import pickle
import random


def file_to_wordset(filename):
    ''' Converts a file with a word per line to a Python set '''
    words = []
    with open(filename, 'r') as f:
        for line in f:
            words.append(line.strip())
    return set(words)


def write_status(i, total):
    ''' Writes status of a process to console '''
    sys.stdout.write('\r')
    sys.stdout.write('Processing %d/%d' % (i, total))
    sys.stdout.flush()


def save_results_to_csv(results, csv_file):
    ''' Save list of type [(tweet_id, positive)] to csv in Kaggle format '''
    with open(csv_file, 'w') as csv:
        csv.write('id,prediction\n')
        for tweet_id, pred in results:
            csv.write(tweet_id)
            csv.write(',')
            csv.write(str(pred))
            csv.write('\n')


def top_n_words(pkl_file_name, N, shift=0):
    """
    Returns a dictionary of form {word:rank} of top N words from a pickle
    file which has a nltk FreqDist object generated by stats.py
    Args:
        pkl_file_name (str): Name of pickle file
        N (int): The number of words to get
        shift: amount to shift the rank from 0.
    Returns:
        dict: Of form {word:rank}
    """
    with open(pkl_file_name, 'rb') as pkl_file:
        freq_dist = pickle.load(pkl_file)
    most_common = freq_dist.most_common(N)
    words = {p[0]: i + shift for i, p in enumerate(most_common)}
    return words


def top_n_bigrams(pkl_file_name, N, shift=0):
    """
    Returns a dictionary of form {bigram:rank} of top N bigrams from a pickle
    file which has a Counter object generated by stats.py
    Args:
        pkl_file_name (str): Name of pickle file
        N (int): The number of bigrams to get
        shift: amount to shift the rank from 0.
    Returns:
        dict: Of form {bigram:rank}
    """
    with open(pkl_file_name, 'rb') as pkl_file:
        freq_dist = pickle.load(pkl_file)
    most_common = freq_dist.most_common(N)
    bigrams = {p[0]: i for i, p in enumerate(most_common)}
    return bigrams


def split_data(tweets, validation_split=0.1):
    """Split the data into training and validation sets
    Args:
        tweets (list): list of tuples
        validation_split (float, optional): validation split %
    Returns:
        (list, list): training-set, validation-set
    """
    index = int((1 - validation_split) * len(tweets))
    random.shuffle(tweets)
    return tweets[:index], tweets[index:]

In [ ]:
!pip install utils

In [ ]:
!python2 /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/code/preprocess.py /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train.csv

Processing 100000/100000
Saved processed tweets to: /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv


In [ ]:
!python2 /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/code/preprocess.py /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/test.csv

Processing 300000/300000
Saved processed tweets to: /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv


In [ ]:
!python2 /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/code/stats.py /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv

Processing 100000/100000
Calculating frequency distribution
Saved uni-frequency distribution to /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist.pkl
Saved bi-frequency distribution to /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist-bi.pkl

[Analysis Statistics]
Tweets => Total: 100000, Positive: 56462, Negative: 43538
User Mentions => Total: 88959, Avg: 0.8896, Max: 12
URLs => Total: 3602, Avg: 0.0360, Max: 4
Emojis => Total: 1001, Positive: 856, Negative: 145, Avg: 0.0100, Max: 5
Words => Total: 1192424, Unique: 50376, Avg: 11.9242, Max: 40, Min: 0
Bigrams => Total: 1092958, Unique: 385024, Avg: 10.9296


In [ ]:
!python2 /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/code/stats.py /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv

Traceback (most recent call last):
  File "/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/code/stats.py", line 63, in <module>
    t_id, if_pos, tweet = line.strip().split(',')
ValueError: need more than 2 values to unpack


## **Baseline**

In [ ]:
!python2 /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/code/baseline.py TRAIN = True

Correct = 65.35%


## **Naive Bayes**

In [ ]:
!python2 /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/code/naivebayes.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7815/10000 = 78.1500 %


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **Maximum Entropy**

In [2]:
!python2 /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/code/maxent-nltk.py TRAIN = True

  ==> Training (1 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.565
         Final          -0.61624        0.756
  -1.010 andyhurleyday==True and label is '0'
   1.000 lancearmstrong==True and label is '0'
   1.000 frenchiebday==True and label is '1'
   1.000 ephesians==True and label is '1'
   1.000 twitterart==True and label is '1'
   1.000 madnessness==True and label is '0'
   1.000 thestreetforce==True and label is '1'
   1.000 atthepool==True and label is '1'
   1.000 twilightnewmoon==True and label is '0'
   1.000 hitech==True and label is '1'
Validation set accuracy:0.0000

Predicting for test data

Saved to maxent.csv


## **Decision Tree**

In [3]:
!python2 /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/code/decisiontree.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 6833/10000 = 68.3300 %


## **Random Forest**

In [4]:
!python2 /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/code/randomforest.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Testing
Processing 1/1
Correct: 7248/10000 = 72.4800 %


## **XGBoost**

In [7]:
import random
import numpy as np
from xgboost import XGBClassifier
from scipy.sparse import lil_matrix
from sklearn.feature_extraction.text import TfidfTransformer

# Performs classification using XGBoost.


FREQ_DIST_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist.pkl'
BI_FREQ_DIST_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist-bi.pkl'
TRAIN_PROCESSED_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv'
TEST_PROCESSED_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv'
TRAIN = True
UNIGRAM_SIZE = 15000
VOCAB_SIZE = UNIGRAM_SIZE
USE_BIGRAMS = True
if USE_BIGRAMS:
    BIGRAM_SIZE = 10000
    VOCAB_SIZE = UNIGRAM_SIZE + BIGRAM_SIZE
FEAT_TYPE = 'frequency'


def get_feature_vector(tweet):
    uni_feature_vector = []
    bi_feature_vector = []
    words = tweet.split()
    for i in xrange(len(words) - 1):
        word = words[i]
        next_word = words[i + 1]
        if unigrams.get(word):
            uni_feature_vector.append(word)
        if USE_BIGRAMS:
            if bigrams.get((word, next_word)):
                bi_feature_vector.append((word, next_word))
    if len(words) >= 1:
        if unigrams.get(words[-1]):
            uni_feature_vector.append(words[-1])
    return uni_feature_vector, bi_feature_vector


def extract_features(tweets, batch_size=500, test_file=True, feat_type='presence'):
    num_batches = int(np.ceil(len(tweets) / float(batch_size)))
    for i in xrange(num_batches):
        batch = tweets[i * batch_size: (i + 1) * batch_size]
        features = lil_matrix((batch_size, VOCAB_SIZE))
        labels = np.zeros(batch_size)
        for j, tweet in enumerate(batch):
            if test_file:
                tweet_words = tweet[1][0]
                tweet_bigrams = tweet[1][1]
            else:
                tweet_words = tweet[2][0]
                tweet_bigrams = tweet[2][1]
                labels[j] = tweet[1]
            if feat_type == 'presence':
                tweet_words = set(tweet_words)
                tweet_bigrams = set(tweet_bigrams)
            for word in tweet_words:
                idx = unigrams.get(word)
                if idx:
                    features[j, idx] += 1
            if USE_BIGRAMS:
                for bigram in tweet_bigrams:
                    idx = bigrams.get(bigram)
                    if idx:
                        features[j, UNIGRAM_SIZE + idx] += 1
        yield features, labels


def apply_tf_idf(X):
    transformer = TfidfTransformer(smooth_idf=True, sublinear_tf=True, use_idf=True)
    transformer.fit(X)
    return transformer


def process_tweets(csv_file, test_file=True):
    """Returns a list of tuples of type (tweet_id, feature_vector)
            or (tweet_id, sentiment, feature_vector)
    Args:
        csv_file (str): Name of processed csv file generated by preprocess.py
        test_file (bool, optional): If processing test file
    Returns:
        list: Of tuples
    """
    tweets = []
    print ('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append((tweet_id, feature_vector))
            else:
                tweets.append((tweet_id, int(sentiment), feature_vector))
            write_status(i + 1, total)
    print ('\n')
    return tweets


if __name__ == '__main__':
    np.random.seed(1337)
    unigrams = top_n_words(FREQ_DIST_FILE, UNIGRAM_SIZE)
    if USE_BIGRAMS:
        bigrams = top_n_bigrams(BI_FREQ_DIST_FILE, BIGRAM_SIZE)
    tweets = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
    if TRAIN:
        train_tweets, val_tweets = split_data(tweets)
    else:
        random.shuffle(tweets)
        train_tweets = tweets
    del tweets
    print ('Extracting features & training batches')
    clf = XGBClassifier(max_depth=25, silent=False, n_estimators=400)
    batch_size = len(train_tweets)
    i = 1
    n_train_batches = int(np.ceil(len(train_tweets) / float(batch_size)))
    for training_set_X, training_set_y in extract_features(train_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
        write_status(i, n_train_batches)
        i += 1
        if FEAT_TYPE == 'frequency':
            tfidf = apply_tf_idf(training_set_X)
            training_set_X = tfidf.transform(training_set_X)
        clf.fit(training_set_X, training_set_y)
    print ('\n')
    print ('Testing')
    if TRAIN:
        correct, total = 0, len(val_tweets)
        i = 1
        batch_size = len(val_tweets)
        n_val_batches = int(np.ceil(len(val_tweets) / float(batch_size)))
        for val_set_X, val_set_y in extract_features(val_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
            if FEAT_TYPE == 'frequency':
                val_set_X = tfidf.transform(val_set_X)
            prediction = clf.predict(val_set_X)
            correct += np.sum(prediction == val_set_y)
            write_status(i, n_val_batches)
            i += 1
        print ('\nCorrect: %d/%d = %.4f %%' % (correct, total, correct * 100. / total))
    else:
        del train_tweets
        test_tweets = process_tweets(TEST_PROCESSED_FILE, test_file=True)
        n_test_batches = int(np.ceil(len(test_tweets) / float(batch_size)))
        predictions = np.array([])
        print ('Predicting batches')
        i = 1
        for test_set_X, _ in extract_features(test_tweets, test_file=True, feat_type=FEAT_TYPE):
            if FEAT_TYPE == 'frequency':
                test_set_X = tfidf.transform(test_set_X)
            prediction = clf.predict(test_set_X)
            predictions = np.concatenate((predictions, prediction))
            write_status(i, n_test_batches)
            i += 1
        predictions = [(str(j), int(predictions[j]))
                       for j in range(len(test_tweets))]
        save_results_to_csv(predictions, 'xgboost.csv')
        print ('\nSaved to xgboost.csv')


Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7677/10000 = 76.7700 %


## **SVM**

In [6]:
!python2 /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/code/svm.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7856/10000 = 78.5600 %


## **Multi-Layer Perceptron**

In [3]:
from keras.models import Sequential, load_model
from keras.layers import Dense
import sys
import random
import numpy as np

# Performs classification using an MLP/1-hidden-layer NN.

FREQ_DIST_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist.pkl'
BI_FREQ_DIST_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist-bi.pkl'
TRAIN_PROCESSED_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv'
TEST_PROCESSED_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv'
xrange=range
TRAIN = True
UNIGRAM_SIZE = 15000
VOCAB_SIZE = UNIGRAM_SIZE
USE_BIGRAMS = False
if USE_BIGRAMS:
    BIGRAM_SIZE = 10000
    VOCAB_SIZE = UNIGRAM_SIZE + BIGRAM_SIZE
FEAT_TYPE = 'frequency'


def get_feature_vector(tweet):
    uni_feature_vector = []
    bi_feature_vector = []
    words = tweet.split()
    for i in xrange(len(words) - 1):
        word = words[i]
        next_word = words[i + 1]
        if unigrams.get(word):
            uni_feature_vector.append(word)
        if USE_BIGRAMS:
            if bigrams.get((word, next_word)):
                bi_feature_vector.append((word, next_word))
    if len(words) >= 1:
        if unigrams.get(words[-1]):
            uni_feature_vector.append(words[-1])
    return uni_feature_vector, bi_feature_vector


def extract_features(tweets, batch_size=500, test_file=True, feat_type='presence'):
    num_batches = int(np.ceil(len(tweets) / float(batch_size)))
    for i in xrange(num_batches):
        batch = tweets[i * batch_size: (i + 1) * batch_size]
        features = np.zeros((batch_size, VOCAB_SIZE))
        labels = np.zeros(batch_size)
        for j, tweet in enumerate(batch):
            if test_file:
                tweet_words = tweet[1][0]
                tweet_bigrams = tweet[1][1]
            else:
                tweet_words = tweet[2][0]
                tweet_bigrams = tweet[2][1]
                labels[j] = tweet[1]
            if feat_type == 'presence':
                tweet_words = set(tweet_words)
                tweet_bigrams = set(tweet_bigrams)
            for word in tweet_words:
                idx = unigrams.get(word)
                if idx:
                    features[j, idx] += 1
            if USE_BIGRAMS:
                for bigram in tweet_bigrams:
                    idx = bigrams.get(bigram)
                    if idx:
                        features[j, UNIGRAM_SIZE + idx] += 1
        yield features, labels


def process_tweets(csv_file, test_file=True):
    tweets = []
    print ('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append((tweet_id, feature_vector))
            else:
                tweets.append((tweet_id, int(sentiment), feature_vector))
            write_status(i + 1, total)
    print('\n')
    return tweets


def build_model():
    model = Sequential()
    model.add(Dense(500, input_dim=VOCAB_SIZE, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    return model


def evaluate_model(model, val_tweets):
    correct, total = 0, len(val_tweets)
    for val_set_X, val_set_y in extract_features(val_tweets, feat_type=FEAT_TYPE, test_file=False):
        prediction = model.predict_on_batch(val_set_X)
        prediction = np.round(prediction)
        correct += np.sum(prediction == val_set_y[:, None])
    return float(correct) / total


if __name__ == '__main__':
    np.random.seed(1337)
    unigrams = top_n_words(FREQ_DIST_FILE, UNIGRAM_SIZE)
    if USE_BIGRAMS:
        bigrams = top_n_bigrams(BI_FREQ_DIST_FILE, BIGRAM_SIZE)
    tweets = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
    if TRAIN:
        train_tweets, val_tweets = split_data(tweets)
    else:
        random.shuffle(tweets)
        train_tweets = tweets
    del tweets
    print ('Extracting features & training batches')
    nb_epochs = 5
    batch_size = 500
    model = build_model()
    n_train_batches = int(np.ceil(len(train_tweets) / float(batch_size)))
    best_val_acc = 0.0
    for j in xrange(nb_epochs):
        i = 1
        for training_set_X, training_set_y in extract_features(train_tweets, feat_type=FEAT_TYPE, batch_size=batch_size, test_file=False):
            o = model.train_on_batch(training_set_X, training_set_y)
            sys.stdout.write('\rIteration %d/%d, loss:%.4f, acc:%.4f' %
                             (i, n_train_batches, o[0], o[1]))
            sys.stdout.flush()
            i += 1
        val_acc = evaluate_model(model, val_tweets)
        print ('\nEpoch: %d, val_acc:%.4f' % (j + 1, val_acc))
        random.shuffle(train_tweets)
        if val_acc > best_val_acc:
            print ('Accuracy improved from %.4f to %.4f, saving model' % (best_val_acc, val_acc))
            best_val_acc = val_acc
            model.save('best_model.h5')
    print ('Testing')
    del train_tweets
    del model
    model = load_model('best_model.h5')
    test_tweets = process_tweets(TEST_PROCESSED_FILE, test_file=True)
    n_test_batches = int(np.ceil(len(test_tweets) / float(batch_size)))
    predictions = np.array([])
    print ('Predicting batches')
    i = 1
    for test_set_X, _ in extract_features(test_tweets, feat_type=FEAT_TYPE, batch_size=batch_size, test_file=True):
        prediction = np.round(model.predict_on_batch(test_set_X).flatten())
        predictions = np.concatenate((predictions, prediction))
        write_status(i, n_test_batches)
        i += 1
    predictions = [(str(j), int(predictions[j]))
                   for j in range(len(test_tweets))]
    save_results_to_csv(predictions, '1layerneuralnet.csv')
    print ('\nSaved to 1layerneuralnet.csv')


Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Iteration 180/180, loss:0.4970, acc:0.7800
Epoch: 1, val_acc:0.7544
Accuracy improved from 0.0000 to 0.7544, saving model
Iteration 180/180, loss:0.4749, acc:0.7980
Epoch: 2, val_acc:0.7651
Accuracy improved from 0.7544 to 0.7651, saving model
Iteration 180/180, loss:0.4254, acc:0.8360
Epoch: 3, val_acc:0.7643
Iteration 180/180, loss:0.4722, acc:0.7720
Epoch: 4, val_acc:0.7601
Iteration 180/180, loss:0.4134, acc:0.8160
Epoch: 5, val_acc:0.7586
Testing
Generating feature vectors
Processing 300000/300000

Predicting batches
Processing 600/600
Saved to 1layerneuralnet.csv


## **Reccurent Neural Network**

In [8]:
import numpy as np
import sys
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import LSTM
from keras.preprocessing.sequence import pad_sequences

# Performs classification using LSTM network.

FREQ_DIST_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist.pkl'
BI_FREQ_DIST_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist-bi.pkl'
TRAIN_PROCESSED_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv'
TEST_PROCESSED_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv'
GLOVE_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/glove-seeds.txt'
dim = 100


def get_glove_vectors(vocab):
    print ('Looking for GLOVE vectors')
    glove_vectors = {}
    found = 0
    with open(GLOVE_FILE, 'r') as glove_file:
        for i, line in enumerate(glove_file):
            write_status(i + 1, 0)
            tokens = line.split()
            word = tokens[0]
            if vocab.get(word):
                vector = [float(e) for e in tokens[1:]]
                glove_vectors[word] = np.array(100)
                found += 1
    print ('\n')
    print ('Found %d words in GLOVE' % found)
    return glove_vectors


def get_feature_vector(tweet):
    words = tweet.split()
    feature_vector = []
    for i in range(len(words) - 1):
        word = words[i]
        if vocab.get(word) is not None:
            feature_vector.append(vocab.get(word))
    if len(words) >= 1:
        if vocab.get(words[-1]) is not None:
            feature_vector.append(vocab.get(words[-1]))
    return feature_vector


def process_tweets(csv_file, test_file=True):
    tweets = []
    labels = []
    print ('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append(feature_vector)
            else:
                tweets.append(feature_vector)
                labels.append(int(sentiment))
            write_status(i + 1, total)
    print ('\n')
    return tweets, np.array(labels)


if __name__ == '__main__':
    train = len(sys.argv) == 1
    np.random.seed(1337)
    vocab_size = 90000
    batch_size = 500
    max_length = 40
    filters = 600
    kernel_size = 3
    vocab = top_n_words(FREQ_DIST_FILE, vocab_size, shift=1)
    glove_vectors = get_glove_vectors(vocab)
    tweets, labels = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
    embedding_matrix = np.random.randn(vocab_size + 1, dim) * 0.01
    for word, i in vocab.items():
        glove_vector = glove_vectors.get(word)
        if glove_vector is not None:
            embedding_matrix[i] = glove_vector
    tweets = pad_sequences(tweets, maxlen=max_length, padding='post')
    shuffled_indices = np.random.permutation(tweets.shape[0])
    tweets = tweets[shuffled_indices]
    labels = labels[shuffled_indices]
    if train:
        model = Sequential()
        model.add(Embedding(vocab_size + 1, dim, weights=[embedding_matrix], input_length=max_length))
        model.add(Dropout(0.4))
        model.add(LSTM(128))
        model.add(Dense(64))
        model.add(Dropout(0.5))
        model.add(Activation('relu'))
        model.add(Dense(1))
        model.add(Activation('sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        filepath = "./models/lstm-{epoch:02d}-{loss:0.3f}-{acc:0.3f}-{val_loss:0.3f}-{val_acc:0.3f}.hdf5"
        checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=True, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.000001)
        print ('model.summary()')
        model.fit(tweets, labels, batch_size=128, epochs=5, validation_split=0.1, shuffle=True, callbacks=[checkpoint, reduce_lr])
    else:
        model = load_model(sys.argv[1])
        print ('model.summary()')
        test_tweets, _ = process_tweets(TEST_PROCESSED_FILE, test_file=True)
        test_tweets = pad_sequences(test_tweets, maxlen=max_length, padding='post')
        predictions = model.predict(test_tweets, batch_size=128, verbose=1)
        results = zip(map(str, range(len(test_tweets))), np.round(predictions[:, 0]).astype(int))
        save_results_to_csv(results, 'lstm.csv')


Looking for GLOVE vectors
Processing 1193514/0

Found 38247 words in GLOVE
Generating feature vectors
Processing 100000/100000



OSError: ignored

## **Convolution Neural Network**

In [10]:
import numpy as np
import sys
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, Flatten
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.sequence import pad_sequences

# Performs classification using CNN.

FREQ_DIST_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist.pkl'
BI_FREQ_DIST_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist-bi.pkl'
TRAIN_PROCESSED_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv'
TEST_PROCESSED_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv'
GLOVE_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/glove-seeds.txt'
dim = 200


def get_glove_vectors(vocab):
    """
    Extracts glove vectors from seed file only for words present in vocab.
    """
    print ('Looking for GLOVE seeds')
    glove_vectors = {}
    found = 0
    with open(GLOVE_FILE, 'r') as glove_file:
        for i, line in enumerate(glove_file):
            write_status(i + 1, 0)
            tokens = line.strip().split()
            word = tokens[0]
            if vocab.get(word):
                vector = [float(e) for e in tokens[1:]]
                glove_vectors[word] = np.array(vector)
                found += 1
    print ('\n')
    return glove_vectors


def get_feature_vector(tweet):
    """
    Generates a feature vector for each tweet where each word is
    represented by integer index based on rank in vocabulary.
    """
    words = tweet.split()
    feature_vector = []
    for i in range(len(words) - 1):
        word = words[i]
        if vocab.get(word) is not None:
            feature_vector.append(vocab.get(word))
    if len(words) >= 1:
        if vocab.get(words[-1]) is not None:
            feature_vector.append(vocab.get(words[-1]))
    return feature_vector


def process_tweets(csv_file, test_file=True):
    """
    Generates training X, y pairs.
    """
    tweets = []
    labels = []
    print ('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append(feature_vector)
            else:
                tweets.append(feature_vector)
                labels.append(int(sentiment))
            write_status(i + 1, total)
    print ('\n')
    return tweets, np.array(labels)


if __name__ == '__main__':
    train = len(sys.argv) == 1
    np.random.seed(1337)
    vocab_size = 90000
    batch_size = 500
    max_length = 40
    filters = 600
    kernel_size = 3
    vocab = top_n_words(FREQ_DIST_FILE, vocab_size, shift=1)
    glove_vectors = get_glove_vectors(vocab)
    tweets, labels = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
    # Create and embedding matrix
    embedding_matrix = np.random.randn(vocab_size + 1, dim) * 0.01
    # Seed it with GloVe vectors
    for word, i in vocab.items():
        glove_vector = glove_vectors.get(word)
        if glove_vector is not None:
            embedding_matrix[200] = glove_vector
    tweets = pad_sequences(tweets, maxlen=max_length, padding='post')
    shuffled_indices = np.random.permutation(tweets.shape[0])
    tweets = tweets[shuffled_indices]
    labels = labels[shuffled_indices]
    if train:
        model = Sequential()
        model.add(Embedding(vocab_size + 1, dim, weights=[embedding_matrix], input_length=max_length))
        model.add(Dropout(0.4))
        model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1))
        model.add(Conv1D(300, kernel_size, padding='valid', activation='relu', strides=1))
        model.add(Conv1D(150, kernel_size, padding='valid', activation='relu', strides=1))
        model.add(Conv1D(75, kernel_size, padding='valid', activation='relu', strides=1))
        model.add(Flatten())
        model.add(Dense(600))
        model.add(Dropout(0.5))
        model.add(Activation('relu'))
        model.add(Dense(1))
        model.add(Activation('sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        filepath = "./models/4cnn-{epoch:02d}-{loss:0.3f}-{acc:0.3f}-{val_loss:0.3f}-{val_acc:0.3f}.hdf5"
        checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=True, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.000001)
        model.fit(tweets, labels, batch_size=128, epochs=8, validation_split=0.1, shuffle=True, callbacks=[checkpoint, reduce_lr])
    else:
        model = load_model(sys.argv[1])
        print ('model.summary()')
        test_tweets, _ = process_tweets(TEST_PROCESSED_FILE, test_file=True)
        test_tweets = pad_sequences(test_tweets, maxlen=max_length, padding='post')
        predictions = model.predict(test_tweets, batch_size=128, verbose=1)
        results = zip(map(str, range(len(test_tweets))), np.round(predictions[:, 0]).astype(int))
        save_results_to_csv(results, 'cnn.csv')


Looking for GLOVE seeds
Processing 1193514/0

Generating feature vectors
Processing 100000/100000



ValueError: ignored

## **Majority Vote Ensemble**

### **Extract-CNN-Feast**

In [12]:
import numpy as np
import sys
from keras.models import load_model, Model
from keras.preprocessing.sequence import pad_sequences

# Extracts dense vector features from penultimate layer of CNN model.

FREQ_DIST_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist.pkl'
BI_FREQ_DIST_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed-freqdist-bi.pkl'
TRAIN_PROCESSED_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/train-processed.csv'
TEST_PROCESSED_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/test-processed.csv'
GLOVE_FILE = '/content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/dataset/glove-seeds.txt'
dim = 200


def get_glove_vectors(vocab):
    print ('Looking for GLOVE seeds')
    glove_vectors = {}
    found = 0
    with open(GLOVE_FILE, 'r') as glove_file:
        for i, line in enumerate(glove_file):
            write_status(i + 1, 0)
            tokens = line.strip().split()
            word = tokens[0]
            if vocab.get(word):
                vector = [float(e) for e in tokens[1:]]
                glove_vectors[word] = np.array(100)
                found += 1
    print ('\n')
    return glove_vectors


def get_feature_vector(tweet):
    words = tweet.split()
    feature_vector = []
    for i in range(len(words) - 1):
        word = words[i]
        if vocab.get(word) is not None:
            feature_vector.append(vocab.get(word))
    if len(words) >= 1:
        if vocab.get(words[-1]) is not None:
            feature_vector.append(vocab.get(words[-1]))
    return feature_vector


def process_tweets(csv_file, test_file=True):
    tweets = []
    labels = []
    print ('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append(feature_vector)
            else:
                tweets.append(feature_vector)
                labels.append(int(sentiment))
            write_status(i + 1, total)
    print ('\n')
    return tweets, np.array(labels)


if __name__ == '__main__':
    train = len(sys.argv) == 1
    np.random.seed(1337)
    vocab_size = 90000
    batch_size = 500
    max_length = 40
    filters = 600
    kernel_size = 3
    vocab = top_n_words(FREQ_DIST_FILE, vocab_size, shift=1)
    glove_vectors = get_glove_vectors(vocab)
    tweets, labels = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
    tweets = pad_sequences(tweets, maxlen=max_length, padding='post')
    shuffled_indices = np.random.permutation(tweets.shape[0])
    tweets = tweets[shuffled_indices]
    labels = labels[shuffled_indices]
    model = load_model(sys.argv[1])
    model = Model(model.layers[0].input, model.layers[-3].output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print ('model.summary()')
    test_tweets, _ = process_tweets(TEST_PROCESSED_FILE, test_file=True)
    test_tweets = pad_sequences(test_tweets, maxlen=max_length, padding='post')
    predictions = model.predict(test_tweets, batch_size=1024, verbose=1)
    np.save('test-feats.npy', predictions)
    predictions = model.predict(tweets, batch_size=1024, verbose=1)
    np.save('train-feats.npy', predictions)
    np.savetxt('train-labels.txt', labels)


Looking for GLOVE seeds
Processing 1193514/0

Generating feature vectors
Processing 100000/100000



OSError: ignored

### **CNN-Feast-SVM**

In [13]:
!python2 /content/drive/MyDrive/Machine_Learning/Twitter-Sentiment-Analysis-main/code/majority-voting.py TRAIN = True

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
